In [1]:
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [2]:
data_folder = 'ml_sec_hw_data/data'

In [3]:
def test_binary_loader(bytes, s):
    l = len(bytes)
    print("bytes length: ", l)

    if l == s:
        # Ha a fájl mérete pontosan s
        x = bytes
    elif l < s:
        # Ha a fájl mérete kisebb mint s, hozzáadunk nullás paddingot
        x = np.pad(bytes, (0, s - l), 'constant', constant_values=(0,))
    else:
        # Ha a fájl mérete nagyobb mint s, átlagoljuk a bájtokat
        group_size = np.ceil(l / s) # csoportok számának kiszámítása
        print("group size: ", group_size)
        # Először hozzáadunk szükséges paddingot
        padding_length = s * group_size - l 
        print("padding length: ", padding_length)
        padded_bytes = np.pad(bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
        print("padding bytes: ", padded_bytes)
        # Átlagolás
        reshaped_bytes = padded_bytes.reshape(-1, int(group_size))
        print("reshaped bytes: \n", reshaped_bytes)
        x = np.mean(reshaped_bytes, axis=1)

    return x

In [4]:
x = test_binary_loader([1,2,3,4], 3)
print(x.shape)
print(x)

bytes length:  4
group size:  2.0
padding length:  2.0
padding bytes:  [1 2 3 4 0 0]
reshaped bytes: 
 [[1 2]
 [3 4]
 [0 0]]
(3,)
[1.5 3.5 0. ]


In [5]:
tensor_x = torch.tensor(x)
print(tensor_x)

tensor_x = tensor_x.unsqueeze(0)
print(tensor_x.shape)

tensor([1.5000, 3.5000, 0.0000], dtype=torch.float64)
torch.Size([1, 3])


In [6]:
def convert_label(label):
    return 'malware' if label == 1 else 'benign'

# 1. feladat

In [7]:
class MalwareDataset(Dataset):
    def __init__(self, data_folder, s=2**14):
        self.data_folder = data_folder
        self.data = []
        self.labels = []
        for label in os.listdir(data_folder): # data/.../train -> malware, benign
            for file in os.listdir(os.path.join(data_folder, label)):
                file_path = os.path.join(data_folder, label, file)
                x = self.preprocess_binary_file(file_path, s)
                self.data.append(x)
                self.labels.append(label)

    def preprocess_binary_file(self, file_path, s):
        # Bináris fájl beolvasása
        with open(file_path, 'rb') as file:
            file_bytes = np.fromfile(file, dtype=np.uint8)

        l = len(file_bytes)

        if l == s:
            # Ha a fájl mérete pontosan s
            x = file_bytes
        elif l < s:
            # Ha a fájl mérete kisebb mint s, hozzáadunk nullás paddingot
            x = np.pad(file_bytes, (0, s - l), 'constant', constant_values=(0,))
        else:
            # Ha a fájl mérete nagyobb mint s, átlagoljuk a bájtokat
            # Először hozzáadunk szükséges paddingot
            padding_length = s * np.ceil(l / s) - l
            padded_bytes = np.pad(file_bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
            # Átlagolás
            x = np.mean(padded_bytes.reshape(-1, int(np.ceil(l / s))), axis=1)

        # Normalizálás [0, 1] tartományba
        x_normalized = x / 255

        return x_normalized

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx]), torch.tensor([1 if self.labels[idx] == 'malware' else 0])
    
def get_loader(data_folder, batch_size=32):
    dataset = MalwareDataset(data_folder)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
train_loader = get_loader(os.path.join(data_folder, 'train'), 32)
test_loader = get_loader(os.path.join(data_folder, 'test'), 32)

In [9]:
data, label = next(iter(train_loader))
print(data.unsqueeze(1).float().shape)
print(label.float().shape)

torch.Size([32, 1, 16384])
torch.Size([32, 1])


In [10]:
class MalwareDetector(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=10, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False),
        )
        self.linear = nn.Linear(in_features=130976, out_features=1, bias=True)
        self.out = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1) # Flatten
        x = self.linear(x)
        x = self.out(x)
        return x

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MalwareDetector().to(device)
bce = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)
model.to(device)

def train(model, train_loader, opt, loss_fn, epochs=10):
    for epoch in range(epochs):
        for data, labels in train_loader:
            data = data.to(device)
            labels = labels.to(device)

            opt.zero_grad()
            out = model(data.unsqueeze(1).float())
            loss = loss_fn(out, labels.float())
            loss.backward()
            opt.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

train(model, train_loader, opt, bce, epochs=10)

Epoch 1/10, Loss: 0.08100762218236923
Epoch 2/10, Loss: 0.09087397903203964
Epoch 3/10, Loss: 0.048470813781023026
Epoch 4/10, Loss: 0.010208791121840477
Epoch 5/10, Loss: 0.008536127395927906
Epoch 6/10, Loss: 0.0025879652239382267
Epoch 7/10, Loss: 0.0028361340519040823
Epoch 8/10, Loss: 0.0002914437500294298
Epoch 9/10, Loss: 0.0005276032024994493
Epoch 10/10, Loss: 0.0007692635990679264


In [12]:
from sklearn.metrics import roc_auc_score, confusion_matrix

def test_model(model, test_loader):
    model.eval()  # Teszt módba állítjuk a modellt
    predictions, true_labels = [], []
    model.to('cpu')
    with torch.no_grad():  # Gradiensek számításának kikapcsolása
        for data, labels in test_loader:
            output = model(data.unsqueeze(1).float())  # Modell alkalmazása
            pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
            predictions.extend(pred_prob)
            true_labels.extend(labels.numpy())
    
    # Bináris osztályozási döntések (0.5 küszöbértékkel)
    pred_labels = [1 if p >= 0.5 else 0 for p in predictions]
    
    # Metrikák kiszámítása
    auc_score = roc_auc_score(true_labels, predictions)
    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels).ravel()
    
    tpr = tp / (tp + fn)  # True Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    fpr = fp / (tn + fp)  # False Positive Rate
    fnr = fn / (tp + fn)  # False Negative Rate
    accuracy = (tp + tn) / (tp + tn + fp + fn)  # Átlagos pontosság
    
    print(f"Accuracy: {accuracy}")
    print(f"TPR: {tpr}")
    print(f"TNR: {tnr}")
    print(f"FPR: {fpr}")
    print(f"FNR: {fnr}")
    print(f"AUC: {auc_score}")
    
    return accuracy, tpr, tnr, fpr, fnr, auc_score


accuracy, tpr, tnr, fpr, fnr, auc = test_model(model, test_loader)

Accuracy: 0.9974358974358974
TPR: 0.9948717948717949
TNR: 1.0
FPR: 0.0
FNR: 0.005128205128205128
AUC: 0.9997633136094674


### Kérdések:
1. Mi a betanított modell átlagos pontossága a teszt adaton ha s = 2
14? : 99.48%
2. Mi a modell TPR, TNR, FPR, FNR, valamint AUC értéke a teszt adaton? Mi állapítható meg ezekből a modell
teljesítményéről? : Magas pontosság, nagyon ritkán téveszt a detektorunk. Felismeri mind a malware-t mind a benign-t. Szinte nincs félreosztályzás


# 2. feladat

## 2.1. feladat

In [ ]:
# malaware-hez tudunk fűzni suffix-et
# ennek tudjuk a hosszát
# csak titszta suffix-et tudunk változtatni
# eretedi bináris -> preprocess -> determine_modifiable_positions -> maszk elemeivel módosítunk -> model bemenet

In [13]:
original_bytes = [1,2,3,4,5,6,7,8,9,10]
adv_suffix = [0] * 5 # ha egy csoport csakis suffix-t tartalmaz akkor a tömörített verzióban is azok lesznek, azaz módosítani tudjuk őket.
adv_bytes = original_bytes + adv_suffix
print("adv_bytes: ", adv_bytes)
x = test_binary_loader(adv_bytes, 6)
print("input: ", x)

adv_bytes:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0]
bytes length:  15
group size:  3.0
padding length:  3.0
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
input:  [2.         5.         8.         3.33333333 0.         0.        ]


In [14]:
def test_determine_modifiable_positions(bytes, s, suffix_length):
    l = len(bytes)
    modifiable_positions = set()
    original_bytes_lenght = l-suffix_length
    print("bytes length: ", l)
    print("original bytes lenght: ", original_bytes_lenght)

    if l >= s:
        # Számítjuk, hány bájt kerül átlagolásra egy vektor elem létrehozásához
        group_size = int(np.ceil(l / s))
        
        print("group size: ", group_size)
       

        # mennyi group kell az eredeti fájlhoz
        original_groups = int(np.ceil(original_bytes_lenght / group_size))
        print("original groups: ", original_groups) 

        # Meghatározzuk, hány bájt kerül hozzáadásra paddingként
        padding_length = int(s * group_size - l)
        print("padding length: ", padding_length)

        padded_bytes = np.pad(bytes, (0, padding_length), 'constant', constant_values=(0,))
        print("padding bytes: ", padded_bytes)
        
        reshaped_bytes = padded_bytes.reshape(-1, group_size)
        
        # Csoportok tartalmazhatnak: (ha márcsak egy másmilyen elem is van mint suffix, akkor az egész csoport nem módosítható)
            # eredeti fájl csoportok
            # (tiszta) suffix csoportok
            # padding csoportok
        
        print("reshaped bytes: \n", reshaped_bytes)

        # mennyi group kell az eredeti fájlhoz
        padding_groups = int(np.ceil(padding_length / group_size))
        print("padding groups: ", padding_groups) 

        total_length = l+padding_length
        total_groups = int(np.ceil(total_length / group_size))
        print("total groups: ", total_groups)

        for i in range(original_groups+1, total_groups-padding_groups+1):
            print(i)
            modifiable_positions.add(i)

        # # Ha a suffix hossza nagyobb, mint a szükséges padding hossza, az utolsó csoportba eső bájtok módosíthatók
        # if suffix_length > padding_length:
        #     modifiable_positions.add(s - 1)  # Az utolsó vektor pozíció módosítható
    
    return modifiable_positions

# Tesztesetek a függvény ellenőrzésére
# Példa bináris fájlok és az 's' paraméter
binaries = [
    ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10] + [0]*4, 6, 4),  # 4 bájt hozzáadva
    ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10] + [0]*5, 6, 5)   # 5 bájt hozzáadva
]

# A determine_modifiable_positions függvény tesztelése
for binary, s, suffix in binaries:
    modifiable_positions = test_determine_modifiable_positions(np.array(binary), s, suffix)
    print(f"Binary length: {len(binary)}, 's' value: {s}, modifiable positions: {modifiable_positions}")


bytes length:  14
original bytes lenght:  10
group size:  3
original groups:  4
padding length:  4
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
padding groups:  2
total groups:  6
Binary length: 14, 's' value: 6, modifiable positions: set()
bytes length:  15
original bytes lenght:  10
group size:  3
original groups:  4
padding length:  3
padding bytes:  [ 1  2  3  4  5  6  7  8  9 10  0  0  0  0  0  0  0  0]
reshaped bytes: 
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10  0  0]
 [ 0  0  0]
 [ 0  0  0]]
padding groups:  1
total groups:  6
5
Binary length: 15, 's' value: 6, modifiable positions: {5}


In [31]:
def determine_modifiable_positions(binary_tensor, s, suffix_length):
    modifiable_positions = set()

    # Original binary length
    original_binary_length = binary_tensor.shape[0]

    # Attach suffix to binary
    suffix_tensor = torch.zeros(suffix_length, dtype=binary_tensor.dtype)
    binary_suffix = torch.cat((binary_tensor, suffix_tensor), dim=0)

    l = binary_suffix.shape[0]
    
    if l >= s:
        group_size = int(np.ceil(l / s))
        
        # How many groups are needed for the original file
        original_groups = int(np.ceil(original_binary_length / group_size))

        # Calculate the padding length needed
        padding_length = int(s * group_size - l)
        
        # How many groups are needed for the padding
        padding_groups = int(np.ceil(padding_length / group_size))
    
        total_length = l + padding_length
        total_groups = int(np.ceil(total_length / group_size))
        
        # Identify modifiable groups
        for i in range(original_groups + 1, total_groups - padding_groups + 1):
            modifiable_positions.add(i)
    
    return modifiable_positions, len(modifiable_positions)

# # Example Usage:
# data = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32)  # Example binary data tensor
# s = 6  # Target vector length
# suffix_length = 5  # Suffix length

# modifiable_positions, num_modifiable_positions, modified_data = determine_modifiable_positions(data, s, suffix_length)
# print(f"Modifiable positions: {modifiable_positions}, \nNumber of modifiable positions: {num_modifiable_positions}")


In [85]:
malware_files = []
victim_malware_path = 'ml_sec_hw_data/data/victim/malware'
for file in os.listdir(victim_malware_path):
    file_path = os.path.join(victim_malware_path, file)
    with open(file_path, 'rb') as file:
        file_bytes = np.fromfile(file, dtype=np.uint8)
        if len(file_bytes) >= 2**14:
            malware_files.append(file_bytes)

In [87]:
def preprocess_binary_file(file_bytes, s):
    l = len(file_bytes)
    padding_length = s * np.ceil(l / s) - l
    padded_bytes = np.pad(file_bytes, (0, int(padding_length)), 'constant', constant_values=(0,))
    x = np.mean(padded_bytes.reshape(-1, int(np.ceil(l / s))), axis=1)
    x_normalized = x / 255

    return x_normalized

In [112]:
s = 2**14
for malware_file in malware_files[:10]:
    test_input = preprocess_binary_file(malware_file, s)
    test_input = torch.tensor(test_input)
    test_input = test_input.unsqueeze(0).unsqueeze(1).float()

    model.eval()
    with torch.no_grad():  # Gradiensek számításának kikapcsolása
        output = model(test_input)  # Modell alkalmazása
        pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
        pred_labels = 1 if pred_prob >= 0.5 else 0

    print(f"Predicted label: {convert_label(pred_labels)}, Prob: {pred_prob}")
    print("True label: ", convert_label(1))

Predicted label: malware, Prob: 0.9997159838676453
True label:  malware
Predicted label: malware, Prob: 0.9999836683273315
True label:  malware
Predicted label: malware, Prob: 0.9998782873153687
True label:  malware
Predicted label: malware, Prob: 0.999765932559967
True label:  malware
Predicted label: malware, Prob: 1.0
True label:  malware
Predicted label: malware, Prob: 0.9988945126533508
True label:  malware
Predicted label: malware, Prob: 0.9024966359138489
True label:  malware
Predicted label: malware, Prob: 0.9998186230659485
True label:  malware
Predicted label: malware, Prob: 0.9997633099555969
True label:  malware
Predicted label: malware, Prob: 0.9999958276748657
True label:  malware


In [125]:
s = 2**14
suffix_percentage = 0.1  # 5% suffix hozzáadása

for malware_file in malware_files[:10]:

    binary = torch.tensor(malware_file).squeeze()
    suffix_length = int(binary.shape[0] * suffix_percentage)
    # Kezdetben nullákat fűzünk hozzá
    suffix_zeros = torch.zeros(suffix_length, dtype=data.dtype)
    modified_data = torch.cat((binary, suffix_zeros), dim=0)

    # Preprocess-t megnézzük, majd kitaláljuk mely pozíciók módosíthatóak
    modified_input = preprocess_binary_file(modified_data, 2**14)

    # Most meghatározzuk, mely pozíciók módosíthatóak
    modifiable_positions, M_len = determine_modifiable_positions(binary, s, suffix_length)

    #print(modifiable_positions)
    #print(M_len)

    for pos in modifiable_positions:
        modified_input[pos] = torch.rand(1)  # Random érték [0, 1] között

    modified_input = torch.tensor(modified_input).unsqueeze(0).unsqueeze(1).float()

    model.eval()
    with torch.no_grad():  # Gradiensek számításának kikapcsolása
        output = model(modified_input)  # Modell alkalmazása
        pred_prob = output.squeeze().numpy()  # Valószínűségek kinyerése
        pred_labels = 1 if pred_prob >= 0.5 else 0

    print(f"Predicted label: {convert_label(pred_labels)}, Prob: {pred_prob}")
    print("True label: ", convert_label(1))

Predicted label: malware, Prob: 0.9999961853027344
True label:  malware
Predicted label: malware, Prob: 0.9739277362823486
True label:  malware
Predicted label: malware, Prob: 0.9978165626525879
True label:  malware
Predicted label: malware, Prob: 0.9234692454338074
True label:  malware
Predicted label: malware, Prob: 1.0
True label:  malware
Predicted label: benign, Prob: 0.30393102765083313
True label:  malware
Predicted label: malware, Prob: 0.993300199508667
True label:  malware
Predicted label: malware, Prob: 0.9304697513580322
True label:  malware
Predicted label: malware, Prob: 0.9996460676193237
True label:  malware
Predicted label: malware, Prob: 1.0
True label:  malware


In [158]:
suffix_percentages = [.05, .1, .15, .2]

def calculate_attack_accuracy(suffix_percentage):
    model.eval()  # Teszt módba állítjuk a modellt
    predictions, true_labels = [], []
    mask_sizes = []
    model.to('cpu')

    for malware_file in malware_files:
        binary = torch.tensor(malware_file).squeeze()
        suffix_length = int(binary.shape[0] * suffix_percentage)
        
        # Kezdetben nullákat fűzünk hozzá
        suffix_zeros = torch.zeros(suffix_length, dtype=data.dtype)
        modified_data = torch.cat((binary, suffix_zeros), dim=0)

        # Eredeti preprocess-t végighatjuk
        modified_input = preprocess_binary_file(modified_data, 2**14)

        # Most meghatározzuk, mely pozíciók módosíthatóak, az eredeti bináris fájl alapján
        modifiable_positions, M_len = determine_modifiable_positions(binary, s, suffix_length)
        mask_sizes.append(M_len)

        # Módosítjuk a megfelelő pozíciókat (ahol csak a adversarial suffix van)
        for pos in modifiable_positions:
            modified_input[pos] = torch.rand(1)  # Random érték [0, 1] között

        # Az inputot megfelelő formátumra alakítjuk
        modified_input = torch.tensor(modified_input).unsqueeze(0).unsqueeze(1).float()
        
        with torch.no_grad():  
            output = model(modified_input)  
            pred_prob = output.squeeze().numpy()  
            predictions.append(pred_prob)
            true_labels.append(1)

    # Bináris osztályozási döntések (0.5 küszöbértékkel)
    pred_labels = [1 if p >= 0.5 else 0 for p in predictions]

    # Attack accuracy és mask size kiszámítása
    attack_acc = (np.array(pred_labels) != np.array(true_labels)).mean()
    mask_size = np.mean(mask_sizes)
    return attack_acc, mask_size

for suffix_percentage in suffix_percentages:
    attack_acc, mask_size = calculate_attack_accuracy(suffix_percentage)
    print(f"Suffix percentage: {suffix_percentage}, Attack Accuracy: {attack_acc*100:.2f}%, Mask size: {mask_size:.0f}")




Suffix percentage: 0.05, Attack Accuracy: 12.24%, Mask size: 703
Suffix percentage: 0.1, Attack Accuracy: 10.20%, Mask size: 1367
Suffix percentage: 0.15, Attack Accuracy: 20.41%, Mask size: 1975
Suffix percentage: 0.2, Attack Accuracy: 16.33%, Mask size: 2524


## 2.2 feladat

In [ ]:
# ismerjük a modell paramétereit, és kitudjuk számolni a gradienseket
# PGD
# x[M] := maszkra szűrt x
# x^t+1[M] = Π[0,1](x^t[M] − ε ⋅ sign(∇x^t[M]lossf(θ, x^t, ytarget)))

In [201]:
def pgd_attack(model, x, y, epsilon, alpha, num_iter, indices_modifiable):
    """Projected Gradient Descent támadás.
    
    Args:
        model: A támadott modell.
        x: A bemeneti adatok tensora.
        y: A cél tensor (a helyes címkék).
        epsilon: A támadás erőssége.
        alpha: Lépésköz minden iterációban.
        num_iter: Az iterációk száma.
        indices_modifiable: A módosítható pozíciók listája vagy tensora.
    
    Returns:
        A módosított bemeneti adatok tensora.
    """
    # Az eredeti adat másolata, amelyet módosítani fogunk
    x_adv = x.clone().detach().requires_grad_(True)

     # A batch és a channel dimenziók figyelembevétele
    indices = torch.zeros_like(x_adv, dtype=torch.bool)  # Először minden értéket False-ra állítunk
    indices[:, :, indices_modifiable] = True  # Csak a módosítható pozíciókat állítjuk True-ra

    
    # Iterációk végrehajtása
    for i in range(num_iter):
        outputs = model(x_adv)
        model.zero_grad()  # Nullázzuk a gradienseket a modellben
        loss = torch.nn.functional.cross_entropy(outputs, y)
        loss.backward()

        with torch.no_grad():
            # Itt a gradiens jele alapján frissítjük az értékeket, de csak a kijelölt True pozíciókon
            x_adv_grad = x_adv.grad.sign()
            x_adv[indices] += alpha * x_adv_grad[indices]
            x_adv = torch.clamp(x_adv, 0, 1)  # Korlátozzuk az értékeket az [0,1] tartományra
            # Az x_adv-nak a ε távolságon belül kell maradnia az eredeti x-től
            x_adv = torch.min(torch.max(x_adv, x - epsilon), x + epsilon)
            x_adv.detach_().requires_grad_(True)  # Leválasztjuk a gradiens számítást, majd újraindítjuk

    
    return x_adv


In [203]:
def calculate_pgd_accuracy(suffix_percentage):

    epsilon = 0.3  # A támadás maximális mértéke (0.03)
    alpha = 0.1  # A támadás lépésköze
    num_iter = 40  # Az iterációk száma
    predictions, true_labels = [], []

    for malware_file in malware_files:
        binary = torch.tensor(malware_file).squeeze()
        suffix_length = int(binary.shape[0] * suffix_percentage)

        # Kezdetben nullákat fűzünk hozzá
        suffix_zeros = torch.zeros(suffix_length, dtype=data.dtype)
        modified_data = torch.cat((binary, suffix_zeros), dim=0)
        modifiable_positions, M_len = determine_modifiable_positions(binary, s, suffix_length)

        modified_input = preprocess_binary_file(modified_data, s)
        x = torch.tensor(modified_input).unsqueeze(0).unsqueeze(1).float()
        y = torch.tensor([1]).unsqueeze(0).float()

        # A PGD támadás végrehajtása
        x_adv = pgd_attack(model, x, y, epsilon, alpha, num_iter, list(modifiable_positions))

        with torch.no_grad():  
            output = model(x_adv)  
            pred_prob = output.squeeze().numpy()  
            predictions.append(pred_prob)
            true_labels.append(1)

    # Bináris osztályozási döntések (0.5 küszöbértékkel)
    pred_labels = [1 if p >= 0.5 else 0 for p in predictions]

    # Attack accuracy és mask size kiszámítása
    attack_acc = (np.array(pred_labels) != np.array(true_labels)).mean()
    
    return attack_acc
    
for suffix_percentage in suffix_percentages:
    attack_acc = calculate_pgd_accuracy(suffix_percentage)
    print(f"Suffix percentage: {suffix_percentage}, Attack Accuracy: {attack_acc*100:.2f}%")



Suffix percentage: 0.05, Attack Accuracy: 0.00%
Suffix percentage: 0.1, Attack Accuracy: 0.00%
Suffix percentage: 0.15, Attack Accuracy: 4.08%
Suffix percentage: 0.2, Attack Accuracy: 8.16%
